In [1]:
!"C:\Program Files\NVIDIA Corporation\NVSMI\nvidia-smi.exe" -L

GPU 0: GeForce RTX 2070 (UUID: GPU-3c1ac771-91d3-c3f2-bedf-547b29fa0086)


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 503392053802898455
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17095265431641531507
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6531337689
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17109867825448517571
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7981862448612374186
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
print(tf.__version__)

2.3.0


In [4]:
policy = mixed_precision.Policy('float32')
mixed_precision.set_policy(policy)
print('Compute dtype: {}'.format(policy.compute_dtype))
print('Variable dtype: {}'.format(policy.variable_dtype))

Compute dtype: float32
Variable dtype: float32


In [5]:
inputs = keras.Input(shape=(784,), name='digits')
if tf.config.list_physical_devices('GPU'):
  print('The model will run with 4096 units on a GPU')
  num_units = 4096
else:
  # Use fewer units on CPUs so the model finishes in a reasonable amount of time
  print('The model will run with 64 units on a CPU')
  num_units = 64
dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

The model will run with 4096 units on a GPU


In [6]:
print('x.dtype: %s' % x.dtype.name)
# 'kernel' is dense1's variable
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)

x.dtype: float32
dense1.kernel.dtype: float32


In [7]:
# INCORRECT: softmax and model output will be float16, when it should be float32
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Outputs dtype: float32


In [8]:
# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Outputs dtype: float32


In [9]:
# The linear activation is an identity function. So this simply casts 'outputs'
# to float32. In this particular case, 'outputs' is already float32 so this is a
# no-op.
outputs = layers.Activation('linear', dtype='float32')(outputs)

In [10]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

In [11]:
initial_weights = model.get_weights()

In [13]:
history = model.fit(x_train, y_train,
                    batch_size=8192,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Epoch 1/5
6/6 [==============================] - 1s 156ms/step - loss: 0.2455 - accuracy: 0.9207 - val_loss: 0.2634 - val_accuracy: 0.9213
Epoch 2/5
6/6 [==============================] - 1s 150ms/step - loss: 0.1998 - accuracy: 0.9386 - val_loss: 0.1962 - val_accuracy: 0.9416
Epoch 3/5
6/6 [==============================] - 1s 149ms/step - loss: 0.3462 - accuracy: 0.8999 - val_loss: 0.1663 - val_accuracy: 0.9551
Epoch 4/5
6/6 [==============================] - 1s 149ms/step - loss: 0.1417 - accuracy: 0.9580 - val_loss: 0.2404 - val_accuracy: 0.9251
Epoch 5/5
6/6 [==============================] - 1s 146ms/step - loss: 0.1256 - accuracy: 0.9612 - val_loss: 0.1284 - val_accuracy: 0.9613
313/313 - 0s - loss: 0.1238 - accuracy: 0.9609
Test loss: 0.12380839884281158
Test accuracy: 0.9609000086784363
